In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform
from tqdm import tqdm
import fiona



# Read the dataset with longitude and latitude of events
data = pd.read_csv(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\FAO_elevation_soil_sum.csv')  # Replace 'events.csv' with the path to your dataset

# Read the areas GeoJSON file
areas_gdf = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\output_3.geojson')



# convert the year_month column to datetime
data['date'] = pd.to_datetime(data['date'])


# Convert the DataFrame into a GeoDataFrame with Points
geometry = [Point(xy) for xy in zip(data['long'], data['lat'])]
data_gdf = gpd.GeoDataFrame(data, geometry=geometry)

# transform the coordinate system
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
x1,y1 = data_gdf['geometry'].x, data_gdf['geometry'].y
x2,y2 = transform(inProj,outProj,x1,y1)
data_gdf['geometry'] = [Point(xy) for xy in zip(x2, y2)]

# unique date
date = areas_gdf['date'].unique()
results = []

for dat in date:

    areas_current_year = areas_gdf[areas_gdf['date'] == dat]  # Assuming the areas are the same for all years

    # filter events within three months 
    events_current_year = data_gdf[(data_gdf['date'] >= dat - pd.DateOffset(months=3)) & (data_gdf['date'] <= dat)]

    # Perform the spatial join
    data_gdf_with_areas = gpd.sjoin(events_current_year, areas_current_year, how='right', op='within')

    results.append(data_gdf_with_areas)

results = pd.concat(results)

# rename date_left to date
results = results.rename(columns={'date_left': 'date'})
# keep only the columns that are needed
results = results[['title','date','battles', 'explosions', 'violence', 'fatalities_battles', 'fatalities_explosions', 'fatalities_violence', 'id', 'estimated_population', 'anl_id', 'overall_phase', 'country', 'year', 'condition', 'phase1_population', 'phase1_percent', 'phase2_population', 'phase2_percent', 'phase3_population', 'phase3_percent', 'phase4_population', 'phase4_percent', 'phase5_population', 'phase5_percent', 'phase3_worse_percentage_manual', 'phase3_plus_phase4', 'phase2_worse_percentage_manual', 'phase2_plus_phase3', 'phase2_plus_phase3_plus_phase4', 'elevation', 'soil', 'price', 'GOSIF_GPP', 'value']]

# summarize the results based on title and date, skip the missing values
results = results.groupby(['title', 'date']).sum().reset_index()

output = results
output_geom = areas_gdf

# keep only geometry and title columns
output_geom = output_geom[['title', 'geometry']]

# remove duplicates accoridng to title and geometry

output_geom = output_geom.drop_duplicates(subset=['title', 'geometry'])


# merge output with output_geom
output = output.merge(output_geom, on='title', how='left')


# transfer output to geodataframe
output = gpd.GeoDataFrame(output)
# save the results
output.to_file('output4.geojson', driver='GeoJSON')

c:\Users\WeilunShi\.virtualenvs\Food_Crisis_and_Insecurity-8fC6jjVv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\WeilunShi\.virtualenvs\Food_Crisis_and_Insecurity-8fC6jjVv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_15532\428834641.py:30: FutureWarning: This function is deprecated. See: https://pyproj